# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 18th 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://c8812d8dfa8b:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/20 02:36:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_airlines = spark.read \
    .option("header", "true") \
    .csv("/opt/spark/work-dir/data/airline/")

df_airlines.show(5)
print(f"Número de registros inicial: {df_airlines.count()}")

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

[Stage 2:>                                                          (0 + 2) / 2]

Número de registros inicial: 300153


In [4]:
# Drop unnecessary columns. Count how many null values the dataset has before/after the cleaning process.

from pyspark.sql.functions import col, when, count, isnull

nulls_before = df_airlines.select([
    count(when(isnull(c) | col(c).isNull(), c)).alias(c) for c in df_airlines.columns
])
nulls_before.show()

[Stage 5:=============================>                             (1 + 1) / 2]

+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



In [5]:
from pyspark.sql.functions import trim

airlines_clean = df_airlines \
    .dropDuplicates(["index"]) \
    .withColumn("airline", trim("airline")) \
    .withColumn("source_city", trim("source_city")) \
    .withColumn("destination_city", trim("destination_city")) \
    .filter(col("price").isNotNull())

print(f"Número de registros después de limpiar: {airlines_clean.count()}")

nulls_after = airlines_clean.select([
    count(when(isnull(c) | col(c).isNull(), c)).alias(c) for c in airlines_clean.columns
])
nulls_after.show()


Número de registros después de limpiar: 300153


[Stage 16:>                                                         (0 + 2) / 2]

+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



In [6]:
#Normalize categorical values: map “zero” → 0, “one” → 1, etc. in stops.

from pyspark.sql.functions import lit

airlines_t1 = airlines_clean.withColumn(
    "stops_numeric",
    when(col("stops") == "zero", lit(0))
    .when(col("stops") == "one", lit(1))
    .when(col("stops") == "two", lit(2))
    .when(col("stops") == "three", lit(3))
    .when(col("stops") == "four", lit(4))
)


In [7]:
from pyspark.sql.functions import col, lit, when, concat_ws

# Transform departure_time and arrival_time to numerical categories (Morning, Afternoon, etc.), then encode as numbers (0=Early_Morning, 1=Morning, etc.).

airlines_t1 = airlines_clean.withColumn(
    "stops_numeric",
    when(col("stops") == "zero", lit(0))
    .when(col("stops") == "one", lit(1))
    .when(col("stops") == "two", lit(2))
    .when(col("stops") == "three", lit(3))
    .when(col("stops") == "four", lit(4))
)

# # Create a new column called route: “Delhi → Mumbai” from source_city and destination_city.

airlines_t2 = airlines_t1.withColumn(
    "route",
    concat_ws(" → ", col("source_city"), col("destination_city"))
)

airlines_t3 = airlines_t2 \
    .withColumn(
        "departure_time_cat",
        when(col("departure_time") == "Early_Morning", lit(0))
        .when(col("departure_time") == "Morning", lit(1))
        .when(col("departure_time") == "Afternoon", lit(2))
        .when(col("departure_time") == "Evening", lit(3))
    ) \
    .withColumn(
        "arrival_time_cat",
        when(col("arrival_time") == "Early_Morning", lit(0))
        .when(col("arrival_time") == "Morning", lit(1))
        .when(col("arrival_time") == "Afternoon", lit(2))
        .when(col("arrival_time") == "Evening", lit(3))
    )

# Add a new column is_expensive: when(price > 6000, True).otherwise(False).
airlines_final = airlines_t3.withColumn(
    "is_expensive",
    when(col("price").cast("double") > 6000, lit(True)).otherwise(lit(False))
) \
    .withColumn("price", col("price").cast("double")) \
    .withColumn("duration", col("duration").cast("double"))

airlines_final.printSchema()
airlines_final.show(5)


root
 |-- index: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- days_left: string (nullable = true)
 |-- price: double (nullable = true)
 |-- stops_numeric: integer (nullable = true)
 |-- route: string (nullable = false)
 |-- departure_time_cat: integer (nullable = true)
 |-- arrival_time_cat: integer (nullable = true)
 |-- is_expensive: boolean (nullable = false)



[Stage 20:=============================>                            (1 + 1) / 2]

+------+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-------+-------------+------------------+------------------+----------------+------------+
| index| airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|  price|stops_numeric|             route|departure_time_cat|arrival_time_cat|is_expensive|
+------+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-------+-------------+------------------+------------------+----------------+------------+
|     0|SpiceJet|SG-8709|      Delhi|       Evening| zero|       Night|          Mumbai|Economy|    2.17|        1| 5953.0|            0|    Delhi → Mumbai|                 3|            NULL|       false|
|   100| Vistara| UK-705|      Delhi| Early_Morning|  one|       Night|          Mumbai|Economy|   12.83|        1|18450.0|            1|    Delhi → Mumbai|                 0| 

In [8]:

# Get the average price per airline.
airlines_final.groupBy("airline").avg("price").show()

# Average duration per route.
airlines_final.groupBy("route").avg("duration").show()

# Minimum and maximum price per airline.
airlines_final.groupBy("airline") \
    .agg({"price": "min", "price": "max"}) \
    .show()

# Count flights by departure_time category.
airlines_final.groupBy("departure_time_cat").count().show()

+---------+------------------+
|  airline|        avg(price)|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
|  Vistara| 30396.53630170735|
+---------+------------------+



+--------------------+------------------+
|               route|     avg(duration)|
+--------------------+------------------+
|Hyderabad → Banga...|12.093316781257894|
|    Mumbai → Kolkata|12.836848119346259|
|    Mumbai → Chennai| 12.66590029615016|
|  Mumbai → Hyderabad|13.263310410697287|
|  Mumbai → Bangalore|11.612022506790714|
|   Bangalore → Delhi| 9.779955655713803|
| Kolkata → Bangalore|13.792946864820731|
|   Hyderabad → Delhi|10.829816600046978|
| Bangalore → Chennai|14.480207488299458|
|    Chennai → Mumbai|12.374656243306948|
|  Bangalore → Mumbai|10.905072262153327|
| Chennai → Bangalore|13.952593562297878|
|      Mumbai → Delhi| 9.818057262475426|
|  Hyderabad → Mumbai| 11.96292329093789|
|   Chennai → Kolkata|14.515774022626285|
|   Kolkata → Chennai|14.774181572222966|
| Kolkata → Hyderabad|13.853107509180667|
|   Delhi → Bangalore|10.354125035683628|
|      Delhi → Mumbai|10.367774216757198|
| Hyderabad → Chennai|13.293238467552772|
+--------------------+------------

+---------+----------+
|  airline|max(price)|
+---------+----------+
|   Indigo|   31952.0|
| SpiceJet|   34158.0|
|Air_India|   90970.0|
|  AirAsia|   31917.0|
| GO_FIRST|   32803.0|
|  Vistara|  123071.0|
+---------+----------+



[Stage 41:=============================>                            (1 + 1) / 2]

+------------------+-----+
|departure_time_cat|count|
+------------------+-----+
|              NULL|49321|
|                 1|71146|
|                 3|65102|
|                 2|47794|
|                 0|66790|
+------------------+-----+



In [9]:
sc.stop()